In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
import os
import zipfile
from google.colab import drive
from keras.preprocessing.image import ImageDataGenerator 
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn import metrics

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Carga de datos

In [ ]:
zip_ref=zipfile.ZipFile('/content/drive/Shareddrives/PovertyModels/data/Data_Rural_vs_Urbano2.zip')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
base_dir='/tmp/'
dev_dir=('/tmp/Data_Rural_vs_Urbano2/Dev')
dev_1_dir=('/tmp/Data_Rural_vs_Urbano2/Dev/Rural')
dev_2_dir=('/tmp/Data_Rural_vs_Urbano2/Dev/Urbano')

print('total de imagenes rurales : ', len(os.listdir(dev_1_dir)))
print('total de imagenes urbanas : ', len(os.listdir(dev_2_dir)))

total de imagenes rurales :  61
total de imagenes urbanas :  49


In [ ]:
base_dir='/tmp/'
test_dir=('/tmp/Data_Rural_vs_Urbano2/Test')
test_1_dir=('/tmp/Data_Rural_vs_Urbano2/Test/Rural')
test_2_dir=('/tmp/Data_Rural_vs_Urbano2/Test/Urbano')

print('total de imagenes rurales : ', len(os.listdir(test_1_dir)))
print('total de imagenes urbanas : ', len(os.listdir(test_2_dir)))

total de imagenes rurales :  61
total de imagenes urbanas :  49


Cargamos el modelo

In [ ]:
longitud, altura = 224,224
modelo1 = '/content/drive/Shareddrives/PovertyModels/Modelos/modelRvU_Resnet.h5'
pesos_modelo1 = '/content/drive/Shareddrives/PovertyModels/Modelos/RvUpesos.h5'
cnn1 = load_model(modelo1)
cnn1.load_weights(pesos_modelo1)

In [ ]:
def predict(file):
  x=load_img(file, target_size=(longitud, altura))
  x=img_to_array(x)
  x= np.expand_dims(x, axis=0)
  array=cnn1.predict(x)
  result=array[0]
  answer=np.argmax(result)

  return answer

In [ ]:
#Rural dev
def listnb3(): 
  counter_0=0
  counter_1=0
  total_r=61 
  lista_dir=[]
  for i in range(1, 62):
    answer_resp=predict('/tmp/Data_Rural_vs_Urbano2/Dev/Rural/Rural'+str(i)+'.jpg')
    if answer_resp==0:
      counter_0 = counter_0+1
    elif answer_resp==1:
      counter_1=counter_1+1
  print('porcentaje de imagenes rurales: ' +str(counter_0/total_r*100)+'%\nporcentaje de imagenes urbanas: '+str(counter_1/total_r*100))

In [ ]:
listnb3()

porcentaje de imagenes rurales: 98.36065573770492%
porcentaje de imagenes urbanas: 1.639344262295082


In [ ]:
#Urbano dev
def listnb4(): 
  counter_0=0
  counter_1=0
  total_r=49 
  lista_dir=[]
  for i in range(1, 50):
    answer_resp=predict('/tmp/Data_Rural_vs_Urbano2/Dev/Urbano/Urbano'+str(i)+'.jpg')
    if answer_resp==0:
      counter_0 = counter_0+1
    elif answer_resp==1:
      counter_1=counter_1+1
  print('porcentaje de imagenes rurales: ' +str(counter_0/total_r*100)+'%\nporcentaje de imagenes urbanas: '+str(counter_1/total_r*100))

In [ ]:
listnb4()

porcentaje de imagenes rurales: 16.3265306122449%
porcentaje de imagenes urbanas: 83.6734693877551


In [ ]:
#Rural Test
def listnb5(): 
  counter_0=0
  counter_1=0
  total_r=61 
  lista_dir=[]
  for i in range(1, 62):
    answer_resp=predict('/tmp/Data_Rural_vs_Urbano2/Test/Rural/Rural'+str(i)+'.jpg')
    if answer_resp==0:
      counter_0 = counter_0+1
    elif answer_resp==1:
      counter_1=counter_1+1
  print('porcentaje de imagenes rurales: ' +str(counter_0/total_r*100)+'%\nporcentaje de imagenes urbanas: '+str(counter_1/total_r*100))

In [ ]:
listnb5()

porcentaje de imagenes rurales: 67.21311475409836%
porcentaje de imagenes urbanas: 32.78688524590164


In [ ]:
#Rural Test
def listnb6(): 
  counter_0=0
  counter_1=0
  total_r=49 
  lista_dir=[]
  for i in range(1, 50):
    answer_resp=predict('/tmp/Data_Rural_vs_Urbano2/Test/Urbano/Urbano'+str(i)+'.jpg')
    if answer_resp==0:
      counter_0 = counter_0+1
    elif answer_resp==1:
      counter_1=counter_1+1
  print('porcentaje de imagenes rurales: ' +str(counter_0/total_r*100)+'%\nporcentaje de imagenes urbanas: '+str(counter_1/total_r*100))

In [ ]:
listnb6()

porcentaje de imagenes rurales: 16.3265306122449%
porcentaje de imagenes urbanas: 83.6734693877551


In [ ]:
dev_data_dir = '/tmp/Data_Rural_vs_Urbano2/Dev'
dev_datagen = ImageDataGenerator()

dev_generator = dev_datagen.flow_from_directory(
    dev_data_dir,
    target_size=(longitud,altura),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)
custom_model=load_model(modelo1)
predictions=custom_model.predict_generator(generator=dev_generator)
y_pred=np.argmax(predictions, axis=1)
y_real=dev_generator.classes

Found 110 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  if sys.path[0] == '':


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_real, y_pred)

array([[60,  1],
       [ 8, 41]])

In [ ]:
print(metrics.classification_report(y_real, y_pred, digits =4))

              precision    recall  f1-score   support

           0     0.8824    0.9836    0.9302        61
           1     0.9762    0.8367    0.9011        49

    accuracy                         0.9182       110
   macro avg     0.9293    0.9102    0.9157       110
weighted avg     0.9242    0.9182    0.9173       110

